In [7]:
import os
import warnings
import random
import pandas as pd
from itertools import combinations
import sys
import networkx as nx
import spacy
import traceback
import pickle

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder


pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

random.seed(100)
warnings.filterwarnings('ignore')


/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [8]:
# !python -m spacy download en_core_web_lg
# nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("en_core_web_lg")

In [9]:
def generateNetwork(sentence_list, ent1_word_idx, ent2_word_idx):
    try:
        sentence = ' '.join(sentence_list)
        doc = nlp(sentence)
        edges = []
        for word_idx, token in enumerate(doc):
            for child in token.children:
                edges.append((token.i,
                          child.i))

        graph = nx.Graph(edges)
        ## If shortest path not found
        try:
            shortest_path_length = nx.shortest_path_length(graph, source=ent1_word_idx, target=ent2_word_idx)
            shortest_path = nx.shortest_path(graph, source=ent1_word_idx, target=ent2_word_idx)
        except: 
            shortest_path_length = -1
            shortest_path = "no_path_found"
        return shortest_path_length, shortest_path
    except:
        print("NETWORK - NO PATH FOUND: ", sentence_list, ent1_word_idx, ent2_word_idx)
        print(traceback.format_exc())
        return -1, "no_path_found"

In [10]:
ENT_TYPES = ["PER", "GPE", "LOC", "ORG"]
def recode_entity_types(value):
    if value.lower() in ["person", "per"]:
        return "PER"
    elif value.lower() in ["organisation", "org"]:
        return "ORG"
    elif value.lower() in ["gpe"]:
        return "GPE"
    elif value.lower() in ["location", "loc"]:
        return "LOC"
    else:
        return "OTHER"

In [11]:
def getIndex(row, sentence, entity):
    try:
        return sentence.index(entity.split(" ")[0]) + 1
    except:
        print("INDEX NOT FOUND - ", sentence, entity, entity.split(" ")[0])
        print(traceback.format_exc())
        return -1

In [12]:
def getFeatures(file, dataset):
    try:
        if dataset not in  ["ace2004", "ace2005", "tacred"]:
            df = pd.read_csv(file, sep="\t", index_col=False, dtype={'text': str, 'entity type': str, 'pos':str , 'tag': str, 'dep': str, 'sent_index': int, 'entity mention ID': str})
        else:    
            df = pd.read_csv(file, index_col=False)  ## For ACE2004 and ACE2005 as sep is different. 
        df = df.rename(columns={"entity_id": "entity mention ID", "sent_idx": "sent_index", "entity_type" : "entity type"})
        df.astype({'text': str, 'entity type': str, 'pos':str , 'dep': str, 'sent_index': int, 'entity mention ID': str})
        df = df.fillna("")
        # filtered_df = df.where(df["entity mention ID"]!="").groupby(["sent_index", "entity mention ID"], as_index=False).agg({'text': ' '.join, 'entity type': '-'.join, 'pos' : '-'.join, 'tag' : '-'.join,'dep': '-'.join})
         ## For ACE2004, ACE2005, tacred, kbp37 as no tag exists
        filtered_df = df.where(df["entity mention ID"]!="").groupby(["sent_index", "entity mention ID"], as_index=False).agg({'text': ' '.join, 'entity type': '-'.join, 'pos' : '-'.join, 'dep': '-'.join}) 
        display(filtered_df)
        ## Calculating entity-entity distances per sentence
        # sent_df = filtered_df.groupby(["sent_index"], as_index=False).agg({'text': list, 'entity type': list, 'pos' : list, 'tag' : list, 'dep': list})
         ##  For ACE2004, ACE2005, tacred, kbp37 as no tag exists
        sent_df = filtered_df.groupby(["sent_index"], as_index=False).agg({'text': list, 'entity type': list, 'pos' : list, 'dep': list})

        text_pairs = []
        ent_type_pairs = []
        pos_pairs = []
        tag_pairs = []
        dep_pairs = []
        
        pos_ignore_list = ["PRON", "PROPN-PART", "DET", "AUX", "CONJ", "CCONJ", "PART","PUNCT", "SYM"]

        for index, row in sent_df.iterrows():
            text_pairs.append(list(combinations(row["text"], r=2)))
            ent_type_pairs.append(list(combinations(row["entity type"], r=2)))
            pos_pairs.append(list(combinations(row["pos"], r=2)))
            dep_pairs.append(list(combinations(row["dep"], r=2)))
            # tag_pairs.append(list(combinations(row["tag"], r=2)))  ## For  ACE2004, ACE2005, tacred, kbp37 as no tag exists

        sent_entity_df = pd.DataFrame()
        sent_entity_df["sent_index"] = sent_df["sent_index"]
        sent_entity_df["entity_pairs"] = text_pairs
        sent_entity_df["ent_type_pairs"] = ent_type_pairs
        sent_entity_df["pos_pairs"] = pos_pairs
        sent_entity_df["dep_pairs"] = dep_pairs
        # sent_entity_df["tag_pairs"] = tag_pairs  ## For ACE2004, ACE2005, tacred, kbp37 as no tag exists 

        final_df = sent_entity_df.set_index('sent_index').apply(lambda x: x.apply(pd.Series).stack()).reset_index().drop('level_1', 1)

        sentences = df.groupby(['sent_index'], as_index=False).agg({'text': list, 'dep': list})
        sentences["sentence_length"] = sentences.apply(lambda x : len(x["text"]), axis=1)
        sentences["root_index"] = sentences.apply(lambda x : int(x["dep"].index("ROOT")), axis=1)
        sentences["root_word"] = sentences.apply(lambda x : x["text"][x["root_index"]], axis=1)
        sentences = sentences.rename(columns={"text": "sentence"})
        final_df = pd.merge(final_df, sentences, on='sent_index', sort=False)
        final_df["entity_1_index"] = final_df.apply(lambda x : getIndex(x, x['sentence'], x['entity_pairs'][0]), axis=1)
        final_df["entity_2_index"] = final_df.apply(lambda x : getIndex(x, x['sentence'], x['entity_pairs'][1]), axis=1)
        final_df["entity_distance"] = final_df.apply(lambda x : x['entity_2_index'] - x['entity_1_index'], axis=1)
        final_df["no_words_before_entity_1"] = final_df["entity_1_index"] - 1 
        final_df["no_words_after_entity_2"] = final_df["sentence_length"] - final_df["entity_2_index"] - 1

        ## Calculating root-entity distances for each sentence
        final_df["entity_1_root_distance"] = final_df.apply(lambda x: x['root_index'] - x['entity_1_index'], axis=1)
        final_df["entity_2_root_distance"] = final_df.apply(lambda x: x['entity_2_index'] - x['root_index'], axis=1)

        # Expand tuples
        final_df[['entity 1 name', 'entity 2 name']] = final_df['entity_pairs'].apply(pd.Series)

        final_df[['entity_type_1', 'entity_type_2']] = final_df['ent_type_pairs'].apply(pd.Series)
        final_df[['entity_pos_1', 'entity_pos_2']] = final_df['pos_pairs'].apply(pd.Series)
        final_df[['entity_dep_1', 'entity_dep_2']] = final_df['dep_pairs'].apply(pd.Series)
        # final_df[['entity_tag_1', 'entity_tag_2']] = final_df['tag_pairs'].apply(pd.Series)  ## For  ACE2004, ACE2005, tacred, kbp37 as no tag exists 
        final_df[["shortest_distance", "shortest_path"]] = final_df.apply(lambda x : generateNetwork(x["sentence"], x["entity_1_index"], x["entity_2_index"]), axis = 1, result_type="expand")

        final_df["entity_type_1"] = final_df["entity_type_1"].apply(lambda x : recode_entity_types(x.split("-")[0]))
        final_df["entity_type_2"] = final_df["entity_type_2"].apply(lambda x : recode_entity_types(x.split("-")[0]))
        final_df["et-label"] = final_df.apply(lambda x : f"{x['entity_type_1']}-{x['entity_type_2']}", axis=1)
        display(final_df)
        final_df = final_df[final_df["entity_type_1"]!= "OTHER"]
        final_df = final_df[final_df["entity_type_2"]!="OTHER"]
        print("After ET filtering", len(final_df))
        final_df = final_df[~final_df['entity_pos_1'].isin(pos_ignore_list)]
        final_df = final_df[~final_df['entity_pos_2'].isin(pos_ignore_list)]
        
        # final_df = final_df.drop([ "entity_pairs", "ent_type_pairs", "pos_pairs", "dep_pairs", "tag_pairs"], axis = 1)  ## For ACE2004, ACE2005, tacred, kbp37 as no tag exists 
        final_df = final_df.drop([ "entity_pairs", "ent_type_pairs", "pos_pairs", "dep_pairs"], axis = 1)
        return final_df
    except:
        print("Failed to execute file : ", file)
        print("Error : ", sys.exc_info())
        print(traceback.format_exc())
        return pd.DataFrame()

In [13]:
def getFeaturesWithlabels(relation_file, tagged_tokens_file, dataset):
    try:
        final_df = getFeatures(tagged_tokens_file, dataset)
        if dataset in ["ace2004", "ace2005", "tacred", "hlt"]:
            relations = pd.read_csv(relation_file)
        else:
            relations = pd.read_csv(relation_file, sep = "\t")
        relations = relations.rename(columns={"ent1_text" : "entity 1 name", "ent2_text" : "entity 2 name", "rel type": "rel_type"})
        result = pd.merge(final_df, relations, on=["entity 1 name", "entity 2 name"], how = "left", sort=False)
        # result.drop_duplicates(subset=["sent_index", "entity 1 name", "entity 2 name"], keep='first', inplace=True, ignore_index=True
        ### Only used for hlt : Remove this codeblock from getFeatures function before uncommenting this. 
        display(final_df)
        display(relations)
#         result["entity_type_1"] = result["entity 1 type"].apply(lambda x : recode_entity_types(x.split("-")[0]))
#         result["entity_type_2"] = result["entity 2 type"].apply(lambda x : recode_entity_types(x.split("-")[0]))
#         result["et-label"] = result.apply(lambda x : f"{x['entity_type_1']}-{x['entity_type_2']}", axis=1)

#         result = result[result["entity_type_1"]!= "OTHER"]
#         result = result[result["entity_type_2"]!="OTHER"]
         
        result["label"] = result["rel_type"].apply(lambda x : 1 if isinstance(x, str) else 0)
        result["label"] = result.apply(lambda x : 0 if x["rel_type"] == "no_relation" else x["label"], axis = 1)
        result = result[result['rel_type'].notna()]
        return result
    except:
        print("Skipping run for : ", relation_file, tagged_tokens_file)
        print(sys.exc_info())
        return pd.DataFrame()
    

In [14]:
# datasets = ["re3d", "ace2004", "ace2005", "tacred", "docred","kbp37", "hlt"]
datasets = ["kbp37"]

ground_truth_path = f"/Users/anishajauhari/Desktop/Sem 4/Independent Study /Dataset/"
tagged_token_path = f"/Users/anishajauhari/Desktop/Sem 4/Independent Study /Dataset/"

In [ ]:
dataset_features = {}
for dataset in datasets:
    print(dataset)
    os.chdir(f"{tagged_token_path}/{dataset}")
    features_with_labels = pd.DataFrame()
    if dataset in ["ace2004", "ace2005"]:
            os.chdir(f"{tagged_token_path}/{dataset}")
    else:
            os.chdir(f"{tagged_token_path}/{dataset}/tagged_tokens")
    for file in os.listdir():
        if file.endswith(".csv"):
            if dataset in ["ace2004", "ace2005"]:
                file = file.split(".csv_")[0]
                tagged_tokens_file = f"{tagged_token_path}/{dataset}/{file}.csv_tagged_tokens.csv"
                relation_file = f"{ground_truth_path}/{dataset}/{file}.csv_gt_relations.csv"
            else:
                tagged_tokens_file = f"{tagged_token_path}/{dataset}/tagged_tokens/{file}"
                relation_file = f"{ground_truth_path}/{dataset}/ground_truth/{file}"
            temp = getFeaturesWithlabels(relation_file, tagged_tokens_file, dataset)
            if len(features_with_labels) == 0:
                features_with_labels = temp
            else:
                features_with_labels = pd.concat([features_with_labels, temp], axis=0)
    ## Keeping relations with greater than 30 support.
    grouped = features_with_labels.groupby(['rel_type'])
    values = grouped["rel_type"].value_counts()
    features_with_labels["support"] = features_with_labels["rel_type"].apply(lambda x : values[x].values[0])
    features_with_labels = features_with_labels[features_with_labels["support"]>30]
    dataset_features[dataset] = features_with_labels
    display(features_with_labels.head(2))

kbp37


,sent_index,entity mention ID,text,entity type,pos,dep
0,0.0,e1_17641,Isle of Wight,--,PROPN-ADP-PROPN,pobj-prep-pobj
1,0.0,e2_17641,Ryde School with Upper Chine,org-org-org-org-org,PROPN-PROPN-ADP-PROPN-PROPN,compound-oprd-prep-compound-pobj
2,1.0,e1_17642,Mars,org,PROPN,nsubj
3,1.0,e2_17642,Maryland,,PROPN,compound
4,2.0,e1_17643,California,,PROPN,compound
...,...,...,...,...,...,...
5929,2964.0,e2_20605,Edward Waters College,per-per-per,PROPN-PROPN-PROPN,compound-compound-pobj
5930,2965.0,e1_20606,NAZA,org,PROPN,compound
5931,2965.0,e2_20606,Malaysia,,PROPN,compound
5932,2966.0,e1_20607,Centrale de Lille,org-org-org,PROPN-ADP-PROPN,nmod-compound-appos


,sent_index,entity_pairs,ent_type_pairs,pos_pairs,dep_pairs,sentence,dep,sentence_length,root_index,root_word,entity_1_index,entity_2_index,entity_distance,no_words_before_entity_1,no_words_after_entity_2,entity_1_root_distance,entity_2_root_distance,entity 1 name,entity 2 name,entity_type_1,entity_type_2,entity_pos_1,entity_pos_2,entity_dep_1,entity_dep_2,shortest_distance,shortest_path,et-label
0,0.0,"(Isle of Wight, Ryde School with Upper Chine)","(--, org-org-org-org-org)","(PROPN-ADP-PROPN, PROPN-PROPN-ADP-PROPN-PROPN)","(pobj-prep-pobj, compound-oprd-prep-compound-pobj)","[After, returning, to, the, U.K., she, attended, the, independent, Upper, Chine, School, in, Shanklin, on, the, Isle, of, Wight, which, subsequently, merged, with, the, independent, Ryde, School, and, was, renamed, Ryde, School, with, Upper, Chine, ), .]","[prep, pcomp, prep, det, pobj, nsubj, ROOT, det, amod, compound, compound, dobj, prep, pobj, prep, det, pobj, prep, pobj, nsubj, advmod, relcl, prep, det, amod, compound, pobj, cc, auxpass, conj, compound, oprd, prep, compound, pobj, punct, punct]",37,6,attended,17,26,9,16,10,-11,20,Isle of Wight,Ryde School with Upper Chine,OTHER,ORG,PROPN-ADP-PROPN,PROPN-PROPN-ADP-PROPN-PROPN,pobj-prep-pobj,compound-oprd-prep-compound-pobj,6,"[17, 16, 14, 6, 21, 22, 26]",OTHER-ORG
1,1.0,"(Mars, Maryland)","(org, )","(PROPN, PROPN)","(nsubj, compound)","[Supported, by, their, own, buying, staff, Mars, purchases, produce, and, seafood, daily, from, local, markets, and, always, includes, local, produce, when, available, from, Maryland, growers, .]","[ROOT, agent, poss, amod, compound, pobj, nsubj, nsubj, ROOT, cc, conj, advmod, prep, amod, pobj, cc, advmod, conj, amod, dobj, advmod, advcl, prep, compound, pobj, punct]",26,0,Supported,7,24,17,6,1,-7,24,Mars,Maryland,ORG,OTHER,PROPN,PROPN,nsubj,compound,7,"[7, 8, 0, 17, 19, 21, 22, 24]",ORG-OTHER
2,2.0,"(California, substance abuse)","(, org-org)","(PROPN, NOUN-NOUN)","(compound, compound-compound)","[The, California, Department, of, Alcohol, and, Drug, Programs, (, ADP, ), is, a, California, state, agency, concerned, with, substance, abuse, prevention, and, treatment, .]","[det, compound, nsubj, prep, pobj, cc, compound, conj, punct, appos, punct, ROOT, det, compound, compound, attr, acl, prep, compound, compound, pobj, cc, conj, punct]",24,11,is,2,19,17,1,4,9,8,California,substance abuse,OTHER,ORG,PROPN,NOUN-NOUN,compound,compound-compound,6,"[2, 11, 15, 16, 17, 20, 19]",OTHER-ORG
3,3.0,"(Aetna, Connecticut)","(org, )","(PROPN, PROPN)","(poss, pobj)","[But, but, Aetna, 's, headquarters, are, in, Connecticut, with, that, great, Senator, Joe, Lieberman, .]","[cc, cc, poss, case, nsubj, ROOT, prep, pobj, prep, det, amod, compound, compound, pobj, punct]",15,5,are,3,8,5,2,6,2,3,Aetna,Connecticut,ORG,OTHER,PROPN,PROPN,poss,pobj,4,"[3, 2, 4, 5, 8]",ORG-OTHER
4,4.0,"(Singapore Airlines, SIA)","(org-org, )","(PROPN-PROPN, PROPN)","(compound-nsubj, appos)","[Singapore, Airlines, (, SIA, ), said, Wednesday, it, would, begin, flying, the, world's, biggest, passenger, plane, the, Airbus, A380, on, its, Melbourne, route, in, late, September, .]","[compound, nsubj, punct, appos, punct, ROOT, npadvmod, nsubj, aux, ccomp, xcomp, det, nmod, amod, compound, dobj, det, compound, appos, prep, poss, compound, pobj, prep, amod, pobj, punct]",27,5,said,1,4,3,0,22,4,-1,Singapore Airlines,SIA,ORG,OTHER,PROPN-PROPN,PROPN,compound-nsubj,appos,1,"[1, 4]",ORG-OTHER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2962,2962.0,"(East Carolina University, Greenville)","(--, org)","(PROPN-PROPN-PROPN, PROPN)","(compound-compound-attr, compound)","[Leo, Warren, Jenkins, (, May, 28, 1913, January, 14, 1989, ), was, the, sixth, president, and, chancellor, of, what, is, now, East, Carolina, University, in, Greenville, North, Carolina, USA, .]","[compound, compound, nsubj, punct, appos, nummod, nummod, npadvmod, nummod, nummod, punct, ROOT,

After ET filtering 0


,sent_index,sentence,dep,sentence_length,root_index,root_word,entity_1_index,entity_2_index,entity_distance,no_words_before_entity_1,no_words_after_entity_2,entity_1_root_distance,entity_2_root_distance,entity 1 name,entity 2 name,entity_type_1,entity_type_2,entity_pos_1,entity_pos_2,entity_dep_1,entity_dep_2,shortest_distance,shortest_path,et-label


...

After ET filtering 0


,sent_index,sentence,dep,sentence_length,root_index,root_word,entity_1_index,entity_2_index,entity_distance,no_words_before_entity_1,no_words_after_entity_2,entity_1_root_distance,entity_2_root_distance,entity 1 name,entity 2 name,entity_type_1,entity_type_2,entity_pos_1,entity_pos_2,entity_dep_1,entity_dep_2,shortest_distance,shortest_path,et-label


,rel_type,rel subtype,entity 1 ID,entity 2 ID,entity 1 mention ID,entity 2 mention ID,entity 1 name,entity 2 name,entity 1 type,entity 2 type,same_sent,distance,pos_pattern,shortest_dep_path,span
0,alternate_names,NaN,e1_15917,e2_15917,e1_15917,e2_15917,Mississippi,MS ANG,NaN,org,True,4.0,PROPN-PROPN-PROPN,2.0,Mississippi Air National Guard ( MS ANG
1,alternate_names,NaN,e1_15918,e2_15918,e1_15918,e2_15918,PSOP,Socialist Party,NaN,org,True,4.0,PROPN-CCONJ-PROPN,2.0,PSOP - Workers and Peasants Socialist Party
2,stateorprovince_of_headquarters,NaN,e1_15919,e2_15919,e1_15919,e2_15919,Integra Bank,Indiana,org,NaN,True,16.0,PROPN-VERB-DET-NOUN-ADP-PROPN-PROPN-PROPN-PROPN-DET-ADJ-NOUN-VERB-ADP-PROPN,7.0,Integra Bank Corporation was the parent of Integra Bank National Association a retail bank headquartered in Evansville Indiana
3,alternate_names,NaN,e1_15920,e2_15920,e1_15920,e2_15920,SA Ambulance Service,SAAS,NaN,org,True,7.0,PROPN-ADP-NUM-CCONJ-ADP-NUM,5.0,SA Ambulance Service Inc in 2005 and in 2008 SAAS
4,stateorprovince_of_headquarters,NaN,e1_15921,e2_15921,e1_15921,e2_15921,college,Kansas,org,NaN,True,18.0,VERB-VERB-ADP-DET-NOUN-ADP-DET-PROPN-PROPN-CCONJ-DET-ADJ-NOUN-NOUN-ADP-ADJ-ADJ,6.0,college is located on the edge of the Flint Hills and the vast wheat fields of south central Kansas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1501,title,NaN,e1_17636,e2_17636,e1_17636,e2_17636,William Gladstone,Prime Minister,per,NaN,NaN,NaN,NaN,NaN,NaN
1502,founded_by,NaN,e1_17637,e2_17637,e1_17637,e2_17637,Intelius,Naveen Jain,org,NaN,NaN,NaN,NaN,NaN,NaN
1503,city_of_headquarters,NaN,e1_17638,e2_17638,e1_17638,e2_17638,London,Camden School for Girls,NaN,org,NaN,NaN,NaN,NaN,NaN
1504,members,NaN,e1_17639,e2_17639,e1_17639,e2_17639,Oneworld,Japan Airlines,org,NaN,NaN,NaN,NaN,NaN,NaN


Skipping run for :  /Users/anishajauhari/Desktop/Sem 4/Independent Study /Dataset//kbp37/ground_truth/dev.csv /Users/anishajauhari/Desktop/Sem 4/Independent Study /Dataset//kbp37/tagged_tokens/dev.csv
(<class 'ValueError'>, ValueError('Columns must be same length as key'), <traceback object at 0x7fc85c8abd70>)


,sent_index,entity mention ID,text,entity type,pos,dep
0,0.0,e1_0,Thom Yorke,per-per,PROPN-PROPN,compound-nsubj
1,0.0,e2_0,Radiohead,,PROPN,pobj
2,1.0,e1_1,Leland High School,org-org-org,PROPN-PROPN-PROPN,compound-compound-nsubj
3,1.0,e2_1,San Jose,-,PROPN-PROPN,compound-compound
4,2.0,e1_2,Ohio University,-,PROPN-PROPN,compound-dobj
...,...,...,...,...,...,...
28545,14272.0,e2_14272,Sheffield Tigers,-,PROPN-PROPN,compound-compound
28546,14273.0,e1_14273,NCAA,,PROPN,compound
28547,14273.0,e2_14273,Virginia Wesleyan College,per-per-per,PROPN-PROPN-PROPN,compound-compound-poss
28548,14274.0,e1_14274,Eredivisie,per,PROPN,nmod


In [ ]:
for data in dataset_features.keys():
    dataset_features[data].to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Features/Latest/features_"+data+".csv")

## Examples

In [ ]:
def print_full():
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', None)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')
print_full()

In [ ]:
# get complete sentence
features_with_labels["sentence"] = features_with_labels["sentence"].apply(lambda x : " ".join(x))

In [ ]:
## For Binary labels
features_with_labels[features_with_labels["label"].apply(lambda x : x==0)][["sentence", "entity 1 name", "entity 2 name", "entity_type_1", "entity_type_2", "label"]]

In [ ]:
features = ["dep", "sentence_length", "root_index", "root_word", "entity_1_index", "entity_2_index", 
            "entity_distance", "no_words_before_entity_1", "no_words_after_entity_2", 
            "entity_1_root_distance", "entity_2_root_distance", 'entity 1 name', 'entity 2 name',
       'entity_type_1', 'entity_type_2', 'entity_pos_1', 'entity_pos_2',
       'entity_dep_1', 'entity_dep_2', 'shortest_distance', 'shortest_path']
examples = ["sentence", "entity 1 name", "entity 2 name", "entity_type_1", "entity_type_2", "et-label", "rel_type", "label", "support"]

In [ ]:
import ast
datasets = ["re3d", "ace2004", "ace2005", "tacred", "docred","kbp37", "hlt"]
path = "/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Features/Latest/"
os.chdir(path)
for file in os.listdir():
    if file.endswith(".csv"):
        dataset = file.split("_")[1][:-4]
        print(file)
        data = pd.read_csv(path + file)
        data["sentence"] = data["sentence"].apply(lambda x : " ".join(ast.literal_eval(x)))
        # data = data[data["label"].apply(lambda x : x==0)]
        data = data[features]
        data.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /ResponsibleRelationExtraction/Features/Features/"+dataset+"_features.csv", index=False)
        
        
        

In [ ]:
df.to_csv("/Users/anishajauhari/Desktop/Sem 4/Independent Study /Examples/"+dataset+"_examples_relations.csv", index=False)